In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'])

/project/6001902/jxchang/TallRid/input/


ModuleNotFoundError: No module named 'xmitgcm'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
import matplotlib.animation as animation
import math

In [ ]:
t = 0
h0 = 140.

f0 = 1.e-4
g = 9.8
u0=0.08

om=2*np.pi/12.4/3600 
alpha = 2e-4 

tR_fname="../indata/TRef.bin"
tRef = np.fromfile(tR_fname)
refSalt=35.
refTemp=tRef[0]
print('refTemp='+ str(refTemp))
print('tRef='+ str(tRef))

xmin = -4
xmax = 4
vmax = 1
vmin = -1
wmax = 0.2
wmin = -0.2
tmax = math.floor(np.amax(tRef))
tmin = math.floor(np.amin(tRef))
numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )
print(tmin)

In [ ]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
ds1=ds1.assign_coords(XC0 = (ds1.XC/1000.-ds1.XC.mean()/1000.))
ds1=ds1.assign_coords(XG0 = (ds1.XG-ds1.XC.mean())/1000.)
ds1=ds1.assign_coords(YC0 = (ds1.YC/1000.-ds1.YC.mean()/1000.))
ds1=ds1.assign_coords(YG0 = (ds1.YG-ds1.YC.mean())/1000.)
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)

In [ ]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

In [ ]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.cos(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,YC=40,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



In [ ]:
# plot temperature
ax2=fig.add_subplot(grid[1,0])
cmap = cm.RdBu_r
levels = np.linspace(vmin, vmax, num=numcolv)
Tlevels = np.linspace(tmin,tmax,7)
print(Tlevels)

norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

cset1 = ax2.contourf(ds1.XG0, ds1.Z, ds1.UVEL.isel(time=t,YC=60), levels , extend='max', **opts)
cset2 = ax2.contour(ds1.XC0, ds1.Z, ds1.THETA.isel(time=t,YC=60), Tlevels , color='Gray')
ax2.clabel(cset2, fontsize=9, inline=1)

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(cset1, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Z [m]')
ax2.set_title('T [oC]', loc='left')
ax2.set(xlim=(xmin, xmax),ylim=(-208,0))
ax2.plot(ds1['XC0'],-ds1.Depth.isel(YC=60),lw=1.5)

In [ ]:
fig
print(ax2.collections)

In [ ]:
tt: int=0
def updateData(tt):

    updatetime = time[tt]
    print(tt)
    ax2.collections = []
    ax2.contourf(ds1.XG0, ds1.Z, ds1.UVEL.isel(time=tt), levels, extend='max', **opts)
    ax2.contour(ds1.XC0, ds1.Z, ds1.THETA.isel(time=tt), Tlevels, color='Gray')
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig.savefig('./figsMag2/summary_TU_xz_t%s.png' %(tt))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 40 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_TU_xz.gif',writer='imagemagick', fps=2)

In [ ]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_TU_xz.gif'))